## Sourcify Database

Download the Sourcify manifest.json file. Sourcify link: https://docs.sourcify.dev/docs/repository/sourcify-database/

In [ ]:
curl -L -O https://repo-backup.sourcify.dev/manifest.json

Modify the manifest.json file to download only files from compiled_contracts, contract_deployments, verified_contracts.

Download the files from the manifest.json using:

In [ ]:
jq -r '.files[].path' manifest.json | xargs -I {} curl -L -O https://repo-backup.sourcify.dev/{}

Move each file to the corresponding folder.

## Python Script manipulating the files to extract the abi in the desired format.

In [1]:
import polars as pl
import glaciers as gl
import os

Remove unecessary columns from each file.

In [5]:
for file in os.listdir('compiled_contracts'):
    if file.endswith('.parquet'):
        df = pl.read_parquet(f'compiled_contracts/{file}')
        df = df.select(
            pl.col('id')
            , pl.col('name')
            , pl.col('fully_qualified_name')
            , pl.col('compilation_artifacts').str.json_path_match('$.abi').alias('abi')
        ).with_columns(
            pl.col('abi').hash().alias('abi_id')
        )
        df.write_parquet(f'compiled_contracts/{file}')

In [6]:
for file in os.listdir('contract_deployments'):
    if file.endswith('.parquet'):
        df = pl.read_parquet(f'contract_deployments/{file}')
        df = df.select(
            pl.col('id')
            , pl.col('chain_id')
            , pl.col('address')
        )
        df.write_parquet(f'contract_deployments/{file}')

In [7]:
for file in os.listdir('verified_contracts'):
    if file.endswith('.parquet'):
        df = pl.read_parquet(f'verified_contracts/{file}')
        df = df.select(
            pl.col('id')
            , pl.col('deployment_id')
            , pl.col('compilation_id')
        )
        df.write_parquet(f'verified_contracts/{file}')

Create dataframes from the files.

In [2]:
abis_df = pl.scan_parquet('compiled_contracts/compiled_contracts_*.parquet').rename({'id': 'compilation_id'}).collect()
contract_ids_df = pl.scan_parquet('verified_contracts/verified_contracts_*.parquet').collect()
deployment_ids_df = pl.scan_parquet('contract_deployments/contract_deployments_*.parquet').rename({'id': 'deployment_id'}).collect()

Join the dataframes.

In [3]:
abis_df = abis_df.join(contract_ids_df, left_on='compilation_id', right_on='compilation_id', how='left')
abis_df = abis_df.join(deployment_ids_df, left_on='deployment_id', right_on='deployment_id', how='left')
abis_df = abis_df.filter(pl.col('address').is_not_null())

Break abis df into 2 files, unique_abis and mapping_contracts_abi_id.

In [7]:
unique_abis_df = abis_df.group_by(['abi_id']).agg(pl.first(['abi', 'address']), pl.len().alias('abi_count')).sort(['abi_id'], descending=True)
unique_abis_df.write_parquet("unique_abis.parquet")
mapping_contracts_abi_id = abis_df.drop(['abi', 'compilation_id', 'deployment_id', 'id']).sort(['chain_id', 'address'])
mapping_contracts_abi_id.write_parquet("mapping_contracts_abi_id.parquet")

For each line in the unique_abis file, read the abi and address, and use glaciers to process the abi, generating a dataframe with each item of the abi.

Concatenate the dataframes and save them in a new file, in steps of 50000 rows.

In [ ]:
abi_df = pl.read_parquet("unique_abis.parquet")
abi_df = abi_df.with_columns(pl.concat_str(pl.lit('0x'), pl.col('address').bin.encode("hex")).alias('address'))

os.makedirs("processed_abis", exist_ok=True)
processed_abi_itens = []
i = 0
for row in abi_df.iter_rows(named=True):
    i += 1
    try:
        result_df = gl.read_new_abi_json(row['abi'], row['address'])
        result_df = result_df.with_columns(pl.lit(row['abi_id']).cast(pl.UInt64).alias('abi_id')).drop(['address'])
        processed_abi_itens.append(result_df)
    except Exception as e:
        print(e)
    if i % 50000 == 0:
        saving_df = pl.concat(processed_abi_itens)
        saving_df.write_parquet(f"processed_abis/processed_abis_{i}.parquet")
        processed_abi_itens = []
saving_df = pl.concat(processed_abi_itens)
saving_df.write_parquet(f"processed_abis/processed_abis_{i}.parquet")

Create a dataframe with the processed abis and break it into 2 files, unique_abis_itens and mapping_item_abi.

In [3]:
processed_abis = pl.scan_parquet("processed_abis/processed_abis_*.parquet").collect()
processed_abis = processed_abis.with_columns(pl.col('full_signature').hash().alias('item_id'))
mapping_item_abi = processed_abis.select(['abi_id', 'item_id']).sort(['item_id'])
mapping_item_abi.write_parquet("mapping_item_abi.parquet")

unique_abis_itens = processed_abis.group_by(['hash', 'full_signature', 'name', 'anonymous', 'num_indexed_args', 'state_mutability', 'item_id']).agg(pl.len().alias('item_count')).sort(['item_count'], descending=True)
unique_abis_itens.write_parquet("unique_abis_itens.parquet")

Filter the events items and join with the mapping_item_abi and mapping_contracts_abi_id to create the ethereum__abis file.

In [2]:
ethereum_chain_id = 1

unique_abis_itens = pl.read_parquet("unique_abis_itens.parquet")
event_abis_itens = unique_abis_itens.filter(pl.col('state_mutability').is_null())
mapping_item_abi = pl.read_parquet("mapping_item_abi.parquet")
mapping_contracts_abi_id = pl.read_parquet("mapping_contracts_abi_id.parquet")
mapping_contracts_abi_id = mapping_contracts_abi_id.filter(pl.col('chain_id') == ethereum_chain_id)

ethereum__abis = event_abis_itens.join(mapping_item_abi, left_on='item_id', right_on='item_id', how='left')
ethereum__abis = ethereum__abis.join(mapping_contracts_abi_id, left_on='abi_id', right_on='abi_id', how='left').rename({'name_right': 'contract_name', 'fully_qualified_name': 'contract_full_name'})
ethereum__abis = ethereum__abis.drop(['abi_id', 'item_id', 'item_count']).filter(pl.col('address').is_not_null())
ethereum__abis = ethereum__abis.with_columns(pl.concat_str(
        pl.lit('0x'), pl.col('hash').bin.encode("hex").str.to_lowercase(),
        pl.lit (' - '), pl.col('full_signature'),
        pl.lit (' - '), pl.lit('0x'), pl.col('address').bin.encode("hex").str.to_lowercase()
    ).alias('id'))
ethereum__abis = ethereum__abis.unique(subset=['id'])

ethereum__abis.write_parquet("sourcify__ethereum__abis__events.parquet")
